Line/Space Function

In [77]:
# Import the needed extensions
import math
import klayout.db as db

def line_cell(tone:str="D",size:float=0.100,pitch:float=0.500,cell_size:float=25,angle:float=0,x2y:float=1,metro_structure:bool = True):

#### Setup ####

    #Initial tone check
    if tone == "D":
            tone = "D"
    elif tone == "C":
            tone = "C"
    else: return TypeError("Error: Tone must be (D)ark or (C)lear)")

    #Create the layout
    layout = db.Layout()
    layout.dbu = 0.001 #um

    #Define initial layers
    l_line = layout.layer(1,0) #line layer, sacrificial

    #Check pitch of the LS array
    pitch_check = size/pitch
    iso = pitch_check < 0.1 #boolean
    bar3 = pitch_check > 0.75 #also boolean

    #Determine naming
    if iso:
        pitch_type = 'iso'
    elif bar3:
        pitch_type = '3bar'
    else:
        pitch_type = f"{pitch}umPitch"

    #Conver angle from degrees to radians
    rad_angle = angle * (math.pi)/(180)


#### Cell creation and shape assignment ####

    #Create the topcell and subsidiary cells
    TopCell = layout.create_cell(f"Initial_cell")
    LineCell = layout.create_cell(f"{size}um_line")

    #Define line dimensions (top and bottom 2x to provide extra length for angled cells)
    l_left = -size/2
    l_right = size/2
    l_2bottom = -cell_size
    l_2top = cell_size

    #Create the main feature shape and insert it into the LineCell cell
    LineCell.shapes(l_line).insert(db.DBox(l_left, l_2bottom, l_right, l_2top))

    #Creates formatting regions
    CellBox = db.DBox((-cell_size/2),-cell_size/2,(cell_size/2),cell_size/2)
    BigBox = db.DBox(-cell_size*1000,-cell_size*1000,cell_size*1000,cell_size*1000)
    BigBox_region = db.Region(BigBox)
    Bar3Line = db.DBox(-1000*1.5*size,-cell_size*1000,1000*1.5*size,cell_size*1000)
    Bar3Line_region = db.Region(Bar3Line)
    DenseLine = db.DBox(-1000*pitch/2,-1000*cell_size,1000*pitch/2,1000*cell_size)
    DenseLine_region = db.Region(DenseLine)


#### Generate the Cell ####

    if iso:
        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell
        if tone == "C":
            iso_region = db.Region(LineCell.shapes(l_line))
            clear_iso = iso_region ^ BigBox_region
            LineCell.clear_shapes()
            LineCell.shapes(l_line).insert(clear_iso)       
        ls_iso = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(ls_iso)

    elif bar3:
        #Create the Bar3Cell
        Bar3Cell =  layout.create_cell(f"{size}um_3bar_structure")
        Bar3Cell.shapes(l_line).insert(db.DBox(-cell_size,-cell_size,3*l_left,cell_size))
        Bar3Cell.shapes(l_line).insert(db.DBox(3*l_right,-cell_size,cell_size,cell_size))
        
        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell
        if tone == "C":
            bar3_region = db.Region(LineCell.shapes(l_line))
            clear_bar3 = bar3_region ^ Bar3Line_region
            LineCell.clear_shapes()
            LineCell.shapes(l_line).insert(clear_bar3)
            Bar3Cell.clear_shapes()
        
        #Add all into the TopCell
        ls_3bar_line = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(ls_3bar_line)    
        ls_3bar_edge = db.DCellInstArray(Bar3Cell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(ls_3bar_edge)

    else:
        #Checks tone and adjusts if needed, then instances a single LineCell into the TopCell
        if tone == "C":
            dense_region = db.Region(LineCell.shapes(l_line))
            clear_dense = dense_region ^ DenseLine_region
            LineCell.clear_shapes()
            LineCell.shapes(l_line).insert(clear_dense)
        
        #Instance a left and right array of the LineCell to span the TopCell region. Trigonometry used to calculate step sizes to preserve pitch for angled arrays.
        ls_array_right = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0),db.DVector((pitch*math.cos(rad_angle)),-pitch*math.sin(rad_angle)),db.DVector(0,0),math.ceil(cell_size/pitch),0)
        ls_array_left = db.DCellInstArray(LineCell,db.DTrans(db.DTrans.M0,0,0),db.DVector((-pitch*math.cos(rad_angle)),pitch*math.sin(rad_angle)),db.DVector(0,0),math.ceil(cell_size/pitch),0)
        TopCell.insert(ls_array_right)
        TopCell.insert(ls_array_left)


#### Add metro structures if applicable ####
    metro_spacing = 8 #um

    if metro_structure:
        MetroCell = layout.create_cell(f"{size}um_line_metro_structure")
        MetroShape = db.DBox(-size/2,-size/2,size/2,size/2)
        MetroCell.shapes(l_line).insert(MetroShape)

        if tone == "C":
             metro_insert = db.DCellInstArray(MetroCell,db.DTrans(db.DTrans.M0,-metro_spacing*math.sin(rad_angle),-metro_spacing*math.cos(rad_angle)),db.DVector(2*metro_spacing*math.sin(rad_angle),2*metro_spacing*math.cos(rad_angle)),db.DVector(0,0),2,0)
        else:
             sqrt_calc = math.sqrt(((size)**2)+(metro_spacing**2))
             extra_angle = -size/metro_spacing
             metro_insert = db.DCellInstArray(MetroCell,db.DTrans(db.DTrans.M0,sqrt_calc*math.sin(rad_angle+extra_angle),sqrt_calc*math.cos(rad_angle+extra_angle)),db.DVector(-2*metro_spacing*math.sin(rad_angle),-2*metro_spacing*math.cos(rad_angle)),db.DVector(2*size*math.cos(rad_angle),-2*size*math.sin(rad_angle)),2,2)

        TopCell.insert(metro_insert)


#### Angle transformations, sliver removal, and output ####
    
    #Apply angle rotation for all cells
    t = db.ICplxTrans(1,angle,0,0,0)
    [layout.cell(i).transform(t) for i in TopCell.each_child_cell()]

    #Clip a new cell that covers just the extents of the defined cell size, and eliminate subcells that contain slivers
    output_cell = layout.clip(TopCell,CellBox)
    listicle = []
    size_check = db.DBox(-size/2,-size/2,size/2,size/2)
    [listicle.append(j) for j in output_cell.each_child_cell() if layout.cell(j).dbbox(l_line).width()<size_check.bbox().width() or layout.cell(j).dbbox(l_line).height()<size_check.bbox().height()]
    [layout.cell(k).prune_cell() for k in listicle]

    #Rename new cell, prune old cell
    if metro_structure:
        output_cell.name = (f"LS_Array_{tone}_{size}umSize_{pitch_type}_{angle}degrees_w_metro")
    else:
        output_cell.name = (f"LS_Array_{tone}_{size}umSize_{pitch_type}_{angle}degrees")
    
    TopCell.prune_cell()

    #Export GDS
    layout.write("LS_Tester.gds")

    return output_cell

line_cell()

Contact Function

In [41]:
# Import the needed extensions
import math
import klayout.db as db

def contact_cell(tone:str="C",size:float=0.100,pitch:float=0.250,cell_size:float=25,angle:float=0,x2y:float=2,metro_structure:bool = True):

#NOTE: Angled mode is currently not configured correctly!

#### Setup ####

    #Initial tone check
    if tone == "D":
            tone = "D"
    elif tone == "C":
            tone = "C"
    else: return TypeError("Error: Tone must be (D)ark or (C)lear)")

    #Create the layout
    layout = db.Layout()
    layout.dbu = 0.001 #um

    #Define initial layers
    l_cont = layout.layer(1,0) #line layer, sacrificial

    #Check pitch of the contact array
    pitch_check = size/pitch
    iso = pitch_check < 0.1 #boolean
    donut = pitch_check > 0.75 #also boolean

    #Determine X and Y pitches, based on x2y value (applies to X-pitch only, Y-pitch is untouched)
    xPitch = (size*x2y) + (pitch-size)
    yPitch = pitch
    xSize = (size*x2y)
    ySize = size

    #Determine naming
    if iso:
        pitch_type = 'iso'
    elif donut:
        pitch_type = 'donut'
    else:
        pitch_type = f"{pitch}umPitch"

    #Convert angle from degrees to radians
    rad_angle = angle * (math.pi)/(180)


#### Cell creation and shape assignment ####

    #Create the topcell and subsidiary cells
    TopCell = layout.create_cell(f"Initial_cell")
    ContCell = layout.create_cell(f"{size}um_{x2y}to1_contact")

    #Define contact dimensions
    c_left = -xSize/2
    c_right = xSize/2
    c_bottom = -ySize/2
    c_top = ySize/2

    #Create the main feature shape and insert it into the ContCell
    cont_iso = db.DBox(c_left, c_bottom, c_right, c_top)
    ContCell.shapes(l_cont).insert(cont_iso)

    #Creates formatting regions for later use
    CellBox = db.DBox((-cell_size/2),-cell_size/2,(cell_size/2),cell_size/2)
    BigBox = db.DBox(-cell_size*1000,-cell_size*1000,cell_size*1000,cell_size*1000)
    BigBox_region = db.Region(BigBox)
    LittleDonut = db.DBox(-1500*(xSize),-1500*(ySize),1500*(xSize),1500*(ySize))
    LittleDonut_region = db.Region(LittleDonut)
    DonutBox = BigBox_region-LittleDonut_region
    DonutBox.break_(4,1)
    DenseCont = db.DBox(-1000*xPitch/2,-1000*yPitch/2,1000*xPitch/2,1000*yPitch/2)
    DenseCont_region = db.Region(DenseCont)


#### Generate the Cell ####

    if iso:
        #Checks tone and adjusts if needed, then instances a single cell into the TopCell
        if tone == "C":
            iso_region = db.Region(ContCell.shapes(l_cont))
            clear_iso = BigBox_region-iso_region
            clear_iso.break_(4,1)
            ContCell.clear_shapes()
            ContCell.shapes(l_cont).insert(clear_iso)       
        cont_iso = db.DCellInstArray(ContCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(cont_iso)

    elif donut:
        #Create the DonutCell
        DonutCell = layout.create_cell(f"{size}um_donut_structure")
        DonutCell.shapes(l_cont).insert(DonutBox)
        
        #Checks tone and adjusts if needed
        if tone == "C":
            donut_region = db.Region(ContCell.shapes(l_cont))
            little_donut = donut_region ^ LittleDonut_region
            ContCell.clear_shapes()
            ContCell.shapes(l_cont).insert(little_donut)
            DonutCell.clear_shapes()
        
        #Add all into the TopCell
        cont_donut_center = db.DCellInstArray(ContCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(cont_donut_center)    
        cont_donut_ring = db.DCellInstArray(DonutCell,db.DTrans(db.DTrans.M0,0,0))
        TopCell.insert(cont_donut_ring)

    else:
        #Checks tone and adjusts if needed
        if tone == "C":
            dense_region = db.Region(ContCell.shapes(l_cont))
            clear_dense = dense_region ^ DenseCont_region
            clear_dense.break_(4,1)
            ContCell.clear_shapes()
            ContCell.shapes(l_cont).insert(clear_dense)
        
        #Instances the contact cell to span the TopCell region. Trigonometry used to calculate step sizes to preserve pitch for angled arrays.
        #NOTE: This "functions" in its current form, but needs further adjusting, especially for holes, to get a good array of angled contacts
        cont_array = db.DCellInstArray(ContCell,db.DTrans(db.DTrans.M0,-xPitch*math.ceil(cell_size/xPitch),-yPitch*math.ceil(cell_size/yPitch)),
                        db.DVector((xPitch*math.cos(rad_angle)),xPitch*math.sin(rad_angle)),db.DVector(yPitch*math.sin(rad_angle),yPitch*math.cos(rad_angle)),
                        2*math.ceil(cell_size/xPitch),2*math.ceil(cell_size/yPitch))
        TopCell.insert(cont_array)

    print(-xPitch*math.ceil(cell_size/xPitch))
    print(-yPitch*math.ceil(cell_size/yPitch))

#### Add metro structures if applicable ####
    metro_spacing = 8 #um

    if metro_structure:
        MetroCell = layout.create_cell(f"{size}um_line_metro_structure")
        if iso:
             MetroShape = db.DPolygon([db.DPoint(-0.5,0),db.DPoint(0,0.5),db.DPoint(0.5,0),db.DPoint(0,-0.5)])
             MetroCell.shapes(l_cont).insert(MetroShape)
             metro_insert = db.DCellInstArray(MetroCell,db.DTrans(db.DTrans.M90,-metro_spacing*math.sin(rad_angle),-metro_spacing*math.cos(rad_angle)),db.DVector(2*metro_spacing*math.sin(rad_angle),2*metro_spacing*math.cos(rad_angle)),db.DVector(0,0),2,0)

             if tone == "C":
                  MetroShape1 = (db.DPolygon([db.DPoint(-500,metro_spacing*1000),db.DPoint(0,500+metro_spacing*1000),db.DPoint(500,metro_spacing*1000),db.DPoint(0,1000*metro_spacing-500)]))
                  MetroShape2 = (db.DPolygon([db.DPoint(-500,-metro_spacing*1000),db.DPoint(0,500-metro_spacing*1000),db.DPoint(500,-metro_spacing*1000),db.DPoint(0,-1000*metro_spacing-500)]))
                  MShape1_Reg = db.Region(MetroShape1)
                  MShape2_Reg = db.Region(MetroShape2)
                  clear_iso = BigBox_region-iso_region
                  new_clear_cont_temp = clear_iso - MShape1_Reg
                  new_clear_cont = new_clear_cont_temp-MShape2_Reg
                  new_clear_cont.break_(5,2)
                  MetroCell.prune_cell()
                  ContCell.shapes(l_cont).clear()
                  ContCell.shapes(l_cont).insert(new_clear_cont)
             else:
                TopCell.shapes(l_cont).insert(metro_insert)

        
        elif not donut:
             if tone == "D":
                Dense_Metro = 1000*db.DBox((-2*xPitch)+xSize/2,(-2*yPitch)-size/2,(-xPitch)-xSize/2,(-2*yPitch)+size/2)
                Dense_Metro_Region = db.Region(Dense_Metro)
                TopCell.shapes(l_cont).insert(Dense_Metro_Region)
             else:
                Dense_Metro = 1000*db.DBox((-2*xPitch)-xSize/2,(-2*yPitch)-size/2,(-2*xPitch)+xSize/2,(-2*yPitch)+size/2)
                Dense_Metro_Region = db.Region(Dense_Metro)
                TopCell.shapes(l_cont).insert(Dense_Metro_Region)


             MetroCell.prune_cell()




#### Angle transformations, sliver removal, and output ####


    #Apply angle rotation for all cells
    t = db.ICplxTrans(1,angle,0,0,0)
    [layout.cell(i).transform(t) for i in TopCell.each_child_cell()]

    #Clip a new cell that covers just the extents of the defined cell size, and eliminate subcells that contain slivers
    output_cell = layout.clip(TopCell,CellBox)
    listicle = []
    size_check = db.DBox(-size,-size,size,size)
    [listicle.append(j) for j in output_cell.each_child_cell() if layout.cell(j).dbbox(l_cont).width()<size_check.bbox().width() or layout.cell(j).dbbox(l_cont).height()<size_check.bbox().height()]
    [layout.cell(k).prune_cell() for k in listicle]

    #Rename new cell, prune old cell
    if metro_structure:
        output_cell.name = (f"Cont_Array_{tone}_{size}umSize_{pitch_type}_{x2y}to1_x2y_{angle}degrees_w_metro")
    else:
        output_cell.name = (f"Cont_Array_{tone}_{size}umSize_{pitch_type}_{x2y}to1_x2y_{angle}degrees")
    
    TopCell.prune_cell()

    #Export GDS
    layout.write("Cont_Tester.gds")

    return output_cell

contact_cell()

-25.2
-25.0


Line with SRAF function

In [1]:
# Import the needed extensions
import math
import klayout
import pya

def lsraf_cell(tone="C",size=300,pitch=100,dimension=5000,angle=45,sraf=60,sraf_num=2):

    #Creates the layout and names it
    layout = pya.Layout()
    Unit = layout.create_cell(f"LSRAF_Array_{tone}_{size}_{pitch}_{angle}_{sraf}_{sraf_num}")

    #Defines the initial layers
    lsraf_outline = layout.layer(11,0) #overlay layer for the cell
    lsraf_line = layout.layer(2,0) #line layer, sacrificial

    #Create the overlay shape
    overlay = Unit.shapes(lsraf_outline).insert(pya.Box((-dimension/2),-dimension/2,(dimension/2),dimension/2))

    #Define vertical line dimensions and both sets of SRAF dimensions
    l_left = -size/2
    l_bottom = -(dimension-(sraf_num+1)*(pitch+sraf))/2
    l_right = size/2
    l_top = (dimension-(sraf_num+1)*(pitch+sraf))/2
    sraf_side_left = -sraf/2
    sraf_side_bottom = l_bottom
    sraf_side_right = sraf/2
    sraf_side_top = l_top
    sraf_end_left = l_left
    sraf_end_bottom = -sraf/2
    sraf_end_right = l_right
    sraf_end_top = sraf/2

    #Create the structure
    line_vert = Unit.shapes(lsraf_line).insert(pya.Box(l_left, l_bottom, l_right, l_top))
    for num in range(-sraf_num,sraf_num+1,1):
        coord = num*pitch
        if coord < 0:
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(l_left+(coord+sraf_side_left),sraf_side_bottom,l_left+(coord+sraf_side_right),sraf_side_top))
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(sraf_end_left,l_bottom+(coord+sraf_end_bottom),sraf_end_right,l_bottom+(coord+sraf_end_top)))
        elif coord > 0:
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(l_right+(coord+sraf_side_left),sraf_side_bottom,l_right+(coord+sraf_side_right),sraf_side_top))
            line_vert = Unit.shapes(lsraf_line).insert(pya.Box(sraf_end_left,l_top+(coord+sraf_end_bottom),sraf_end_right,l_top+(coord+sraf_end_top)))

    #Does the angle transformation for the lines for rotations
    t = pya.ICplxTrans(1,angle,0,0,0)
    Unit.shapes(lsraf_line).transform(t)

    #Removes polygons extending beyond the overlay layer by only including those within the overlay region (& statement)
    r1 = pya.Region(Unit.shapes(lsraf_line))
    r2 = pya.Region(Unit.shapes(lsraf_outline))
    r_and = r1 & r2
    
    #Flips tone using XOR if tone is C instead of D
    if tone == "C":
        r_and = r_and ^ r2
    
    #Adds new layer with finished cell
    l_diff = layout.layer(1,0)
    r_diff = Unit.shapes(l_diff).insert(r_and)

    #Removes sacrificial layer
    l_remove = layout.delete_layer(1)

    #Export GDS
    layout.write(f"LSRAF_test_t{tone}_s{size}_p{pitch}_l{dimension}_a{angle}_sraf{sraf}_numsrafs{sraf_num}.gds")

lsraf_cell()

Spiral PCell

In [11]:
# Import the needed extensions
import math
import klayout
import pya

#Going to make this a new PCell for Klayout to use, I essentially copied this from here: (https://www.youtube.com/watch?v=2e6DuFj0Xws)
class LSCell(pya.PCellDeclarationHelper):

    def __init__(self):
    
        #Initiallizes the super class
        super(LSCell, self).__init__()

        #Declare the parameters
        self.param("l", self.TypeLayer, "Layer", default = pya.LayerInfo(1,0)) #sets up the layer
        self.param("n", self.TypeInt, "Number of points", default = 64)
        self.param("inner_r", self.TypeDouble, "Inner Radius", default = 1) #defines starting point 
        self.param("outer_r", self.TypeDouble, "Outer Radius", default = 10) #defines the extent of the cell!
        self.param("width", self.TypeDouble, "Width", default = 1)
        self.param("spacing", self.TypeDouble, "Spacing", default = 1)
        self.param("inner_handle", self.TypeShape, "", default = pya.DPoint(0,0)) #handle used to change radius independent of radius parameter
        self.param("outer_handle", self.TypeShape, "", default = pya.DPoint(0,0)) 

        #Additional hidden parameters for whether the radius has changed or the "s" handle has been moved
        self.param("inner_memory", self.TypeDouble, "Inner Memory", default = 0.0, hidden = True)
        self.param("outer_memory", self.TypeDouble, "Outer Memory", default = 0.0, hidden = True)

    def display_text_impl(self):
        #Provides a descriptive text for the cell
        return "Spiral(L = " + str(self.l) + ", Outer radius = " + ('%.3f' % self.outer_r) + ")"
    
    def coerce_parameters_impl(self):

        #We employ coerce_parameters_impl to decide whether the handle or the numeric paramter has changed
        #(by comparing against the effective radius ru)
        #and set ru to the effective radius. We also update the numerical value or the shape,
        #depending on which one has not changed

        inner_handle_radius = None
        outer_handle_radius = None

        if isinstance(self.inner_handle, pya.DPoint):
            #if inner radius manually changed on layout by user, calculates value, computes distance in microns from 0,0 point
            inner_handle_radius = self.inner_handle.distance(pya.DPoint(0,0))
        if isinstance(self.outer_handle, pya.DPoint):
            #if inner radius manually changed on layout by user, calculates value, computes distance in microns from 0,0 point
            inner_handle_radius = self.outer_handle.distance(pya.DPoint(0,0))
        
        if abs(self.inner_r - self.inner_memory) < 1e-6 and abs(self.outer_r - self.outer_memory) < 1e-6: #case where either handle changes
            self.inner_memory = inner_handle_radius
            self.inner_r = inner_handle_radius
            self.outer_memory = outer_handle_radius
            self.outer_r = outer_handle_radius
        else: #case where the parameter changes
            self.inner_memory = self.inner_r
            self.inner_handle = pya.DPoint(-self.inner_r,0)
            self.outer_memory = self.outer_r
            self.outer_handle = pya.DPoint(-self.outer_r,0)
        
        #n must be larger than or equal to 4
        if self.n <= 4:
            self.n = 4
    
    def can_create_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: we can use any shape which has a finite bounding box
        return self.shape.is_box() or self.shape.is_polygon() or self.shape.is_path()
    
    def parameters_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: we set the inner and outer radii based on the bounding box of the shape being used
        self.inner_r = self.shape.bbox().width() * self.layout.dbu / 4
        self.outer_r = self.shape.bbox().width() * self.layout.dbu / 2
        self.l = self.layout.get_info(self.layer) #puts the new spiral shape on the same layer as the shape being converted
    
    def transformation_from_shape_impl(self):
        #Implement the "Create PCell from shape" protocol: places the spiral in the same position as the converted shape
        return pya.Trans(self.shape.bbox().center())
    
    def produce_impl(self):
        #This is the main part of the function, where the shape is actually formed/created

        #Fetch the parameters, and convert to the database units being used in the layout
        inner_r_dbu = self.inner_r / self.layout.dbu
        outer_r_dbu = self.outer_r / self.layout.dbu

        #Set up array for the points to be placed for the spiral
        pts = []
        #The change in angle (in radians) being stepped across to drive the spiral
        da = math.pi * 2 / self.n
        #The change in radius being stepped across to expand the spiral.
        #After one rotation, want the spiral to have expanded by one width and one spacing.
        dr = (self.width+self.spacing)/self.n/self.layout.dbu

        #Initial conditions for spiral loop.
        current_radius = inner_r_dbu
        current_angle = 0

        #While loop used to fill out the spiral
        while current_radius < outer_r_dbu:
            #Adds a new point to the pts array for each while loop pass
            #Cosine and sine calculations converting the angle in radians to x,y coordinates
            pts.append(pya.Point.from_dpoint(pya.Dpoint(current_radius * math.cos(current_angle), current_radius * math.sin(current_angle))))
            #Steps the radius up
            current_radius += dr
            #Steps up the current angle.
            #Modulus of 2pi is being used to avoid angle buildup after 2pi is reached, reverting back to 0 instead
            current_angle = (current_angle+da)%(math.pi*2)
        
        #Creates the shape
        self.cell.shapes(self.l_layer).insert(pya.Path(pts, self.width/self.layout.dbu))

class MyLib(pya.Library):
    #The library where we put the PCell into
    def __init__(self):
        
        #Set the description
        self.description = "My Library"

        #Create the PCell declarations
        self.layout().register_pcell("Spiral", LSCell())

        #Register with the nam "MyLib"
        self.register("MyLib")

#Instantiate and register the library
MyLib()